<a href="https://colab.research.google.com/github/trungduyen0220/text-classification-assignment/blob/master/Assignment1_NguyenThiMyDuyen_MSE13028.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%tensorflow_version 2.x

# Dataset

In [137]:
!rm -f train.csv test.csv
!wget https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/train.csv
!wget https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/test.csv
!wget http://nlp.stanford.edu/data/wordvecs/glove.6B.zip

--2020-09-06 10:41:14--  https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/train.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10400749 (9.9M) [text/plain]
Saving to: ‘train.csv’

train.csv           100%[===================>]   9.92M  26.3MB/s    in 0.4s    

2020-09-06 10:41:15 (26.3 MB/s) - ‘train.csv’ saved [10400749/10400749]

--2020-09-06 10:41:15--  https://raw.githubusercontent.com/minhpqn/NLP-Notes/master/data/arxiv/test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1133794 (1.1M) [text/plain]
Saving

In [193]:
import pandas as pd
import numpy as np
import csv
import zipfile
import glob
from string import digits

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df.head()

,id,title,abstract,category
0,1901.05003,Quantum Teleportation-Inspired Algorithm for S...,We show that low-depth random quantum circui...,quant-ph
1,1903.08905,RAP-Net: Recurrent Attention Pooling Networks ...,The response selection has been an emerging ...,cs
2,1907.11533,Combinatorial protein-protein interactions on ...,Scaffold proteins organize cellular processe...,q-bio
3,1711.07582,CVXR: An R Package for Disciplined Convex Opti...,CVXR is an R package that provides an object...,stat
4,1910.05833,Analytical Solution of (2+1) Dimensional Dirac...,"In this article, we studied the system of (2...",quant-ph


In [141]:
def read_csv(filename):
    abstract = []
    category = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile )

        for row in csvReader:
            abstract.append(row[2])
            category.append(row[3])

    category = pd.factorize(category)[0]
    X = np.asarray(abstract)
    Y = np.asarray(category)

    return X, Y

X_train, Y_train = read_csv('train.csv')
X_test, Y_test = read_csv('test.csv')

for index in range(10):
    print(str(train_df['category'][index]) + " " + str(Y_train[index + 1]))

quant-ph 1
cs 2
q-bio 3
stat 4
quant-ph 1
q-bio 3
hep-ph 5
quant-ph 1
quant-ph 1
hep-ph 5


In [142]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y
  
Y_oh_train = convert_to_one_hot(Y_train, C = 11)
Y_oh_test = convert_to_one_hot(Y_test, C = 11)
idx = 1
print(f"Label index {Y_train[idx]} in one-hot encoding format is {Y_oh_train[idx]}")
idx = 4
print(f"Label index {Y_train[idx]} in one-hot encoding format is {Y_oh_train[idx]}")

Label index 1 in one-hot encoding format is [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Label index 4 in one-hot encoding format is [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]


In [143]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map
    

file = glob.glob('glove.6B.*.txt', recursive=True)
if len(file) == 0:
  with zipfile.ZipFile("glove.6B.zip","r") as zip_ref:
    zip_ref.extractall()


In [156]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.50d.txt')
word = "quantum"
idx = 289846
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(idx) + "th word in the vocabulary is", index_to_word[idx])

the index of quantum in the vocabulary is 297181
the 289846th word in the vocabulary is potatoes


In [211]:
notdefined = []
def sentence_to_avg(sentence, word_to_vec_map):

    # Step 1: Split sentence into list of lower case words (≈ 1 line)

    words = preformat(sentence).lower().split()
    # Initialize the average word vector, should have the same shape as your word vectors.
    avg = np.zeros((50,)) # <Gema> one dimensional array with 50 elements (we use pretrained 50-dimensional GloVe embeddings)
    
    # Step 2: average the word vectors. You can loop over the words in the list "words".
    for w in words:
        try:
          word_to_vec_map[w]
        except:
          notdefined.append(w)
        else:
          avg = np.zeros((50,))+word_to_vec_map[w]
    
    avg = avg/len(words)
    
    return avg
# avg = sentence_to_avg("Morrocan couscous is my favorite dish", word_to_vec_map)
# print("avg = \n", avg)
# print(X_train[1])

In [187]:
def model(X, Y, word_to_vec_map, learning_rate = 0.01, num_iterations = 1500):
    
    np.random.seed(1)

    # Define number of training examples
    m = Y.shape[0]                          # number of training examples
    n_y = 11                                 # number of classes  
    n_h = 50                                # dimensions of the GloVe vectors 
    
    # Initialize parameters using Xavier initialization
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    # Convert Y to Y_onehot with n_y classes
    Y_oh = convert_to_one_hot(Y, C = n_y) 
    
    # Optimization loop
    for t in range(num_iterations):                       # Loop over the number of iterations
        for i in range(m):                                # Loop over the training examples
            
            # Average the word vectors of the words from the i'th training example
            avg = sentence_to_avg(X[i], word_to_vec_map)

            # Forward propagate the avg through the softmax layer
            z = np.dot(W,avg)+b
            a = np.exp(z - np.max(z))/(np.exp(z - np.max(z))).sum() # softmax

            # Compute cost using the i'th training label's one hot representation and "A" (the output of the softmax)
            cost = -np.sum(Y_oh[i]*np.log(a))
            
            # Compute gradients 
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz

            # Update parameters with Stochastic Gradient Descent
            W = W - learning_rate * dW
            b = b - learning_rate * db
        
        if t % 100 == 0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            pred = predict(X, Y, W, b, word_to_vec_map)

    return pred, W, b

In [214]:
def preformat(sentence):
  sentence = sentence.replace(',','')
  sentence = sentence.replace('.','')
  sentence = sentence.replace('-',' ')
  sentence = sentence.replace('*',' ')
  sentence = sentence.replace('(',' ')
  sentence = sentence.replace(')',' ')
  sentence = sentence.translate(str.maketrans('', '', digits))
  return sentence

In [217]:
def predict(X, Y, W, b, word_to_vec_map):
    """
    Given X (sentences) and Y (emoji indices), predict emojis and compute the accuracy of your model over the given set.
    
    Arguments:
    X -- input data containing sentences, numpy array of shape (m, None)
    Y -- labels, containing index of the label emoji, numpy array of shape (m, 1)
    
    Returns:
    pred -- numpy array of shape (m, 1) with your predictions
    """
    m = X.shape[0]
    pred = np.zeros((m, 1))
    
    for j in range(m):                       # Loop over training examples
        
        # Split jth test example (sentence) into list of lower case words
        words = preformat(X[j]).lower().split()
        
        # Average words' vectors
        avg = np.zeros((50,))
        for w in words:
            try:
              word_to_vec_map[w]
            except:
              notdefined.append(w)
            else:
              avg = np.zeros((50,))+word_to_vec_map[w]
        avg = avg/len(words)

        # Forward propagation
        Z = np.dot(W, avg) + b
        A = np.exp(Z - np.max(Z))/(np.exp(Z - np.max(Z))).sum() #softmax
        
        pred[j] = np.argmax(A)
        
    print("Accuracy: "  + str(np.mean((pred[:] == Y.reshape(Y.shape[0],1)[:]))))
    
    return pred

In [ ]:
pred, W, b = model(X_train, Y_train, word_to_vec_map)

Epoch: 0 --- cost = 2.3106762489844694
Accuracy: 0.09998889012331963
Epoch: 100 --- cost = 2.2699095097146333
Accuracy: 0.18342406399288969
